In [ ]:
import os
import numpy as np
import librosa
import soundfile as sf

def preprocess_audio(audio_path, target_shape=(128, 128), duration_sec=2, target_sr=8000):
    audio_data, sample_rate = librosa.load(audio_path, sr=target_sr)
    num_samples = int(duration_sec * sample_rate)
    
  
    if len(audio_data) > num_samples:
        audio_data = audio_data[:num_samples]
    else:
        audio_data = np.pad(audio_data, (0, num_samples - len(audio_data)))
    
    # Calculate STFT [128, 128]
    n_fft = 256 
    hop_length = int(len(audio_data) / target_shape[1])
    
    # Convert audio to frequency domain using STFT
    stft_result = np.abs(librosa.stft(audio_data, n_fft=n_fft, hop_length=hop_length))
    
    
    if stft_result.shape[0] > target_shape[0]:
        stft_result = stft_result[:target_shape[0], :]
    else:
        stft_result = np.pad(stft_result, ((0, target_shape[0] - stft_result.shape[0]), (0, 0)))
    
    
    if stft_result.shape[1] > target_shape[1]:
        stft_result = stft_result[:, :target_shape[1]]
    else:
        stft_result = np.pad(stft_result, ((0, 0), (0, target_shape[1] - stft_result.shape[1])))
    
    
    audio_data = stft_result.T
    
    return audio_data

def process_and_save_audio(input_folder, output_folder, target_shape=(128, 128), duration_sec=2, target_sr=8000):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith('.wav'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            preprocessed_audio = preprocess_audio(input_path, target_shape, duration_sec, target_sr)
            audio_flattened = preprocessed_audio.flatten()
            
            
            sf.write(output_path, audio_flattened, target_sr)
            print(f"Processed and saved: {output_path}")

input_folder = '/home/masense/projects/ai8x-training/data/AudioAutoencoder/raw/fan/id_00/orig'
output_folder = '/home/masense/projects/ai8x-training/data/AudioAutoencoder/raw/fan/id_00/normal'
process_and_save_audio(input_folder, output_folder)